# Multi-Asset Cryptocurrency Trading Bot with Enhanced Performance

# Improvements:
# - Multi-asset training (BTC, ETH, XRP)
# - Date range: 2019-01-01 to present
# - Enhanced risk management
# - Better position sizing
# - Improved win rate strategies


## 1. Import Libraries and Setup

In [ ]:
import ccxt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import json
from typing import Dict, List, Tuple, Optional
import requests
from scipy import stats
import talib
import time
warnings.filterwarnings('ignore')

In [ ]:
# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Enhanced Data Collection for Multiple Assets

In [ ]:
def download_crypto_data_with_date(symbol='BTC/USDT', timeframe='1h', start_date='2019-01-01'):
    """Download crypto data from specific date to present - FIXED for full data"""
    exchange = ccxt.binance({
        'rateLimit': 1200,
        'enableRateLimit': True,
    })
    
    # Convert start date to timestamp
    start_timestamp = exchange.parse8601(start_date + 'T00:00:00Z')
    end_timestamp = exchange.milliseconds()
    
    print(f"Downloading {symbol} {timeframe} data from {start_date}...")
    all_ohlcv = []
    
    current_timestamp = start_timestamp
    batch_count = 0
    
    while current_timestamp < end_timestamp:
        try:
            # Fetch batch
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, current_timestamp, limit=1000)
            
            if not ohlcv:
                break
                
            # Add to results
            all_ohlcv.extend(ohlcv)
            batch_count += 1
            
            # Update timestamp for next batch
            last_timestamp = ohlcv[-1][0]
            
            # Check if we've reached the end
            if last_timestamp >= end_timestamp or len(ohlcv) < 1000:
                break
                
            # Move to next batch (add 1ms to avoid duplicate)
            current_timestamp = last_timestamp + 1
            
            # Progress update
            print(f"Downloaded {len(all_ohlcv)} candles (batch {batch_count})...", end='\r')
            
            # Rate limiting
            time.sleep(exchange.rateLimit / 1000)
            
        except Exception as e:
            print(f"\nError downloading batch {batch_count}: {e}")
            time.sleep(5)
            # Retry from last successful timestamp
            if all_ohlcv:
                current_timestamp = all_ohlcv[-1][0] + 1
            else:
                # If first batch failed, wait and retry
                time.sleep(10)
                continue
    
    # Create DataFrame
    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.drop_duplicates(subset=['timestamp']).reset_index(drop=True)
    df['symbol'] = symbol
    
    # Calculate total days of data
    if len(df) > 0:
        days_of_data = (df['datetime'].max() - df['datetime'].min()).days
        print(f"\nDownloaded {len(df)} candles for {symbol} ({days_of_data} days of data)")
    else:
        print(f"\nNo data downloaded for {symbol}")
    
    return df

In [ ]:
def download_multiple_assets(symbols=['BTC/USDT', 'ETH/USDT', 'XRP/USDT'], 
                           timeframe='1h', start_date='2019-01-01'):
    """Download data for multiple cryptocurrency pairs"""
    all_data = {}
    
    for symbol in symbols:
        try:
            data = download_crypto_data_with_date(symbol, timeframe, start_date)
            all_data[symbol] = data
            time.sleep(2)  # Be nice to the API
        except Exception as e:
            print(f"Failed to download {symbol}: {e}")
            
    return all_data

In [ ]:
# Download data for multiple assets
print("Downloading multi-asset data...")
symbols = ['BTC/USDT', 'ETH/USDT', 'XRP/USDT']
multi_asset_data = download_multiple_assets(symbols, timeframe='1h', start_date='2019-01-01')

## 3. Enhanced Feature Engineering for Multi-Asset

In [ ]:
def calculate_advanced_features(df, asset_name='BTC'):
    """Calculate only essential features - optimized version"""
    df = df.copy()
    
    # === Core Price Features (5) ===
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    df['high_low_range'] = (df['high'] - df['low']) / df['close']
    df['close_position'] = (df['close'] - df['low']) / (df['high'] - df['low'] + 1e-8)
    
    # === Essential Volatility (2) ===
    df['volatility'] = df['returns'].rolling(20).std()
    df['atr'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    
    # === Key Moving Averages (6) ===
    df['sma_20'] = talib.SMA(df['close'], timeperiod=20)
    df['sma_50'] = talib.SMA(df['close'], timeperiod=50)
    df['ema_20'] = talib.EMA(df['close'], timeperiod=20)
    
    # Price relative to MA
    df['price_to_sma20'] = (df['close'] - df['sma_20']) / df['sma_20']
    df['price_to_sma50'] = (df['close'] - df['sma_50']) / df['sma_50']
    df['ma_divergence'] = (df['sma_20'] - df['sma_50']) / df['sma_50']
    
    # === Critical Momentum Indicators (4) ===
    df['rsi'] = talib.RSI(df['close'], timeperiod=14)
    df['macd'], df['macd_signal'], df['macd_hist'] = talib.MACD(df['close'])
    
    # === Bollinger Bands (2) ===
    df['bb_upper'], df['bb_middle'], df['bb_lower'] = talib.BBANDS(df['close'], timeperiod=20)
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-8)
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']
    
    # === Volume Analysis (2) ===
    df['volume_ratio'] = df['volume'] / df['volume'].rolling(20).mean()
    df['volume_trend'] = talib.OBV(df['close'], df['volume']) / 1e6  # Normalize
    
    # === Market Structure (3) ===
    df['support'] = df['low'].rolling(20).min()
    df['resistance'] = df['high'].rolling(20).max()
    df['sr_ratio'] = (df['close'] - df['support']) / (df['resistance'] - df['support'] + 1e-8)
    
    # === Multi-timeframe (2) ===
    df['returns_4h'] = df['close'].pct_change(4)
    df['returns_24h'] = df['close'].pct_change(24)
    
    # === Asset identifier ===
    asset_map = {'BTC': 0, 'ETH': 1, 'XRP': 2}
    df['asset_id'] = asset_map.get(asset_name.split('/')[0], 0)
    
    # Clean data
    df = df.dropna()
    df = df.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df

In [ ]:
# Apply feature engineering to all assets
for symbol, data in multi_asset_data.items():
    asset_name = symbol.split('/')[0]
    multi_asset_data[symbol] = calculate_advanced_features(data, asset_name)
    print(f"Features calculated for {symbol}: {multi_asset_data[symbol].shape}")

# Combine all assets for training
combined_data = pd.concat(list(multi_asset_data.values()), ignore_index=True)
combined_data = combined_data.sort_values('datetime').reset_index(drop=True)
print(f"Combined data shape: {combined_data.shape}")

# Enhanced feature columns
feature_columns = [
    'returns', 'log_returns', 'high_low_range', 'close_position',
    'volatility', 'atr',
    'price_to_sma20', 'price_to_sma50', 'ma_divergence',
    'rsi', 'macd', 'macd_signal', 'macd_hist',
    'bb_position', 'bb_width',
    'volume_ratio', 'volume_trend',
    'sr_ratio',
    'returns_4h', 'returns_24h',
    'asset_id'
]


## 4. Enhanced XGBoost Direction Predictor

In [ ]:
class EnhancedDirectionPredictor:
    """XGBoost with GPU support and optimized parameters"""
    
    def __init__(self, lookback=24, prediction_horizon=4):
        self.lookback = lookback
        self.prediction_horizon = prediction_horizon
        self.models = {}
        self.best_threshold = 0.5
        
    def prepare_data(self, df, feature_cols):
        """Prepare data with better labeling strategy"""
        X, y = [], []
        
        for i in range(self.lookback, len(df) - self.prediction_horizon):
            # Features
            X.append(df[feature_cols].iloc[i-self.lookback:i].values.flatten())
            
            # Improved target: consider high/low within prediction window
            current_price = df['close'].iloc[i]
            
            # Check future price movement including high/low
            future_highs = df['high'].iloc[i+1:i+self.prediction_horizon+1].max()
            future_lows = df['low'].iloc[i+1:i+self.prediction_horizon+1].min()
            future_close = df['close'].iloc[i + self.prediction_horizon]
            
            # Calculate potential gains/losses
            max_gain = (future_highs - current_price) / current_price
            max_loss = (future_lows - current_price) / current_price
            final_return = (future_close - current_price) / current_price
            
            # Better labeling: consider risk/reward
            if max_gain > 0.015 and max_loss > -0.01:  # Good risk/reward
                y.append(1)
            elif max_loss < -0.015:  # High risk
                y.append(0)
            else:
                y.append(0 if final_return < 0 else 1)
        
        return np.array(X), np.array(y)
    
    def train_ensemble(self, X_train, y_train, X_val, y_val, n_models=3):
        """Train with GPU if available"""
        print("Training XGBoost with GPU support...")
        
        # Check for GPU
        try:
            import GPUtil
            gpus = GPUtil.getGPUs()
            use_gpu = len(gpus) > 0
        except:
            use_gpu = False
        
        print(f"GPU available: {use_gpu}")
        
        # Base parameters
        base_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'tree_method': 'gpu_hist' if use_gpu else 'hist',
            'predictor': 'gpu_predictor' if use_gpu else 'cpu_predictor',
            'use_label_encoder': False,
            'random_state': 42
        }
        
        # Find optimal threshold
        best_f1 = 0
        for thresh in [0.4, 0.45, 0.5, 0.55, 0.6]:
            params = {
                **base_params,
                'max_depth': 6,
                'learning_rate': 0.05,
                'n_estimators': 200,
                'subsample': 0.8,
                'colsample_bytree': 0.8
            }
            
            model = xgb.XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                     early_stopping_rounds=20, verbose=False)
            
            pred_proba = model.predict_proba(X_val)[:, 1]
            pred = (pred_proba > thresh).astype(int)
            
            # Calculate F1 score
            from sklearn.metrics import f1_score
            f1 = f1_score(y_val, pred)
            
            if f1 > best_f1:
                best_f1 = f1
                self.best_threshold = thresh
        
        print(f"Best threshold: {self.best_threshold} with F1: {best_f1:.3f}")
        
        # Train final ensemble with best parameters
        for i in range(n_models):
            print(f"Training model {i+1}/{n_models}")
            
            params = {
                **base_params,
                'max_depth': np.random.randint(4, 8),
                'learning_rate': np.random.uniform(0.03, 0.1),
                'n_estimators': np.random.randint(150, 300),
                'subsample': np.random.uniform(0.7, 0.9),
                'colsample_bytree': np.random.uniform(0.7, 0.9),
                'min_child_weight': np.random.randint(1, 5),
                'gamma': np.random.uniform(0, 0.2),
                'reg_alpha': np.random.uniform(0, 0.1),
                'reg_lambda': np.random.uniform(1, 2)
            }
            
            model = xgb.XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                     early_stopping_rounds=30, verbose=False)
            
            self.models[f'model_{i}'] = model
    
    def predict_proba(self, X):
        """Ensemble prediction"""
        predictions = []
        for model in self.models.values():
            predictions.append(model.predict_proba(X)[:, 1])
        return np.mean(predictions, axis=0)
    
    def predict(self, X):
        """Get trading signals"""
        proba = self.predict_proba(X)
        predictions = np.zeros(len(proba))
        
        # Strong signals only
        predictions[proba > self.best_threshold + 0.1] = 1  # Strong bullish
        predictions[proba < self.best_threshold - 0.1] = -1  # Strong bearish
        
        return predictions

## 5. Enhanced PPO Implementation

In [ ]:
class EnhancedPPOAgent:
    """Enhanced PPO with better exploration and learning"""
    
    def __init__(self, state_dim, action_dim, lr=3e-4):
        self.action_dim = action_dim
        
        # Networks
        self.network = ActorCriticNetwork(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)
        
        # Enhanced PPO parameters
        self.gamma = 0.995  # Increased for longer-term thinking
        self.gae_lambda = 0.97
        self.clip_epsilon = 0.2
        self.value_coef = 0.5
        self.entropy_coef = 0.02  # Increased for more exploration
        self.max_grad_norm = 0.5
        
        # Adaptive learning rate
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='max', factor=0.5, patience=10
        )
        
        # Memory
        self.states = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.log_probs = []
        self.dones = []
        
        # Performance tracking
        self.episode_returns = []
        
    def act(self, state, xgb_signal=None, valid_actions=None, explore_rate=0.1):
        """Select action with enhanced exploration"""
        # Validate state
        state = np.nan_to_num(state, 0.0)
        state = np.clip(state, -10, 10)
        
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        
        with torch.no_grad():
            action_probs, value = self.network(state_tensor)
        
        # Ensure valid probabilities
        action_probs = action_probs.cpu()
        
        if valid_actions is None:
            valid_actions = list(range(self.action_dim))
        
        # Mask invalid actions
        mask = torch.zeros(1, self.action_dim)
        for action in valid_actions:
            mask[0, action] = 1
        
        # Apply mask
        action_probs = action_probs * mask
        
        # Renormalize
        if action_probs.sum() > 0:
            action_probs = action_probs / action_probs.sum(dim=-1, keepdim=True)
        else:
            action_probs = mask / mask.sum()
        
        # Enhanced XGBoost signal incorporation
        if xgb_signal is not None and not np.isnan(xgb_signal):
            signal_strength = min(abs(xgb_signal), 1.0)  # Cap signal strength
            
            if xgb_signal > 0 and 1 in valid_actions:  # Bullish
                action_probs[0, 1] *= (1 + 0.5 * signal_strength)
                if 2 in valid_actions:
                    action_probs[0, 2] *= (1 - 0.3 * signal_strength)
            elif xgb_signal < 0 and 2 in valid_actions:  # Bearish
                if 1 in valid_actions:
                    action_probs[0, 1] *= (1 - 0.3 * signal_strength)
                action_probs[0, 2] *= (1 + 0.5 * signal_strength)
            
            # Renormalize
            action_probs = action_probs / action_probs.sum(dim=-1, keepdim=True)
        
        # Exploration vs exploitation
        if np.random.random() < explore_rate:
            # Random valid action
            action = np.random.choice(valid_actions)
        else:
            # Sample from distribution
            dist = Categorical(action_probs)
            action = dist.sample().item()
        
        # Store for training
        self.states.append(state)
        self.actions.append(action)
        if value.dim() == 0:
            self.values.append(value.cpu().item())
        else:
            self.values.append(value[0].cpu().item() if len(value) > 0 else 0.0)
        
        # Calculate log prob for the taken action
        dist = Categorical(action_probs)
        self.log_probs.append(dist.log_prob(torch.tensor(action)).item())
        
        return action
    
    def update(self, epochs=10):
        """Update with adaptive learning"""
        if len(self.states) < 64:  # Increased minimum batch
            return
        
        # Compute advantages and returns
        advantages, returns = self.compute_gae()
        
        if advantages.numel() == 0:
            return
        
        # Calculate average return for scheduler
        avg_return = np.mean(self.rewards[-100:]) if len(self.rewards) >= 100 else 0
        self.scheduler.step(avg_return)
        
        # Rest of update logic remains the same...
        # [Previous update code here]
        
    def compute_gae(self):
        """Compute Generalized Advantage Estimation"""
        # Ensure all lists have same length
        min_length = min(len(self.rewards), len(self.values), len(self.dones))
        
        if min_length == 0:
            return torch.FloatTensor([]).to(device), torch.FloatTensor([]).to(device)
        
        # Trim to same length
        rewards = self.rewards[:min_length]
        values = self.values[:min_length]
        dones = self.dones[:min_length]
        
        advantages = []
        advantage = 0
        
        for t in reversed(range(len(rewards))):
            if t == len(rewards) - 1:
                next_value = 0
            else:
                next_value = values[t + 1]
            
            delta = rewards[t] + self.gamma * next_value * (1 - dones[t]) - values[t]
            advantage = delta + self.gamma * self.gae_lambda * (1 - dones[t]) * advantage
            advantages.insert(0, advantage)
        
        advantages = torch.FloatTensor(advantages).to(device)
        returns = advantages + torch.FloatTensor(values).to(device)
        
        # Normalize advantages
        if advantages.numel() > 0:
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        return advantages, returns
    
    def store_reward(self, reward, done):
        """Store reward and done flag"""
        self.rewards.append(reward)
        self.dones.append(done)

In [ ]:
class ActorCriticNetwork(nn.Module):
    """Enhanced Actor-Critic network"""
    
    def __init__(self, state_dim, action_dim, hidden_dim=512):  # Larger network
        super().__init__()
        
        # Shared layers with more capacity
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim // 2)
        
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim // 2)
        
        # Actor head
        self.actor_fc = nn.Linear(hidden_dim // 2, hidden_dim // 4)
        self.actor_out = nn.Linear(hidden_dim // 4, action_dim)
        
        # Critic head
        self.critic_fc = nn.Linear(hidden_dim // 2, hidden_dim // 4)
        self.critic_out = nn.Linear(hidden_dim // 4, 1)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.2)
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize weights properly"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, state):
        # Check for NaN in input
        if torch.isnan(state).any():
            state = torch.nan_to_num(state, 0.0)
        
        # Shared network with residual connection
        x = F.relu(self.ln1(self.fc1(state)))
        x = self.dropout(x)
        
        x2 = F.relu(self.ln2(self.fc2(x)))
        x2 = self.dropout(x2)
        
        x3 = F.relu(self.ln3(self.fc3(x2)))
        x3 = self.dropout(x3)
        
        # Actor
        actor = F.relu(self.actor_fc(x3))
        action_logits = self.actor_out(actor)
        action_probs = F.softmax(action_logits, dim=-1) + 1e-8
        
        # Critic
        critic = F.relu(self.critic_fc(x3))
        value = self.critic_out(critic).squeeze(-1)
        
        return action_probs, value

## 6. Enhanced Trading Environment

In [ ]:
class EnhancedCryptoFuturesEnv:
    """Fixed environment with proper high/low liquidation checks"""
    
    def __init__(self, data, initial_balance=10000, fee_rate=0.0005, 
                 max_leverage=10, liquidation_margin=0.05):
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        self.fee_rate = fee_rate
        self.max_leverage = max_leverage
        self.liquidation_margin = liquidation_margin
        
        # Position sizing
        self.min_position_pct = 0.02   # 2% minimum
        self.max_position_pct = 0.10   # 10% maximum  
        self.base_position_pct = 0.05  # 5% base
        
        # Risk management
        self.stop_loss_pct = 0.02      # 2% stop loss
        self.take_profit_pct = 0.03    # 3% take profit
        self.trailing_stop_pct = 0.015 # 1.5% trailing
        
        self.feature_columns = feature_columns
        self.action_space = 4
        
        self.reset()
    
    def _check_intrabar_liquidation(self, entry_price, high, low, position_type, leverage):
        """Check if position would be liquidated based on high/low"""
        if position_type == 'long':
            # For long: check if low price hits liquidation
            worst_pnl = (low - entry_price) / entry_price
            if worst_pnl * leverage <= -self.liquidation_margin:
                return True, low
        else:  # short
            # For short: check if high price hits liquidation
            worst_pnl = (entry_price - high) / entry_price
            if worst_pnl * leverage <= -self.liquidation_margin:
                return True, high
        
        return False, None
    
    def _execute_action(self, action, exit_reason=None):
        """Execute trading action with proper P&L calculation"""
        current_price = self.data['close'].iloc[self.current_step]
        current_high = self.data['high'].iloc[self.current_step]
        current_low = self.data['low'].iloc[self.current_step]
        reward = 0
        
        # Validate action
        valid_actions = self.get_valid_actions()
        if action not in valid_actions:
            return -0.1
        
        if action == 0:  # HOLD
            if self.position is None:
                reward = -0.0005
            else:
                # Check intrabar liquidation while holding
                liquidated, liq_price = self._check_intrabar_liquidation(
                    self.entry_price, current_high, current_low, 
                    self.position, self.leverage
                )
                
                if liquidated:
                    # Force liquidation
                    self._close_position(liq_price, 'liquidation')
                    reward = -1.0
                else:
                    # Normal holding reward
                    if self.position == 'long':
                        pnl_pct = (current_price - self.entry_price) / self.entry_price
                    else:
                        pnl_pct = (self.entry_price - current_price) / self.entry_price
                    
                    reward = 0.01 if pnl_pct > 0 else -0.002
        
        elif action in [1, 2] and self.position is None:  # Open position
            # Calculate position size
            position_size = self._calculate_position_size(action)
            fee = position_size * self.fee_rate
            
            if position_size + fee <= self.balance:
                self.balance -= fee
                self.position = 'long' if action == 1 else 'short'
                self.entry_price = current_price
                self.entry_step = self.current_step
                self.position_size = position_size
                
                # Set initial stops
                if self.position == 'long':
                    self.stop_loss_price = current_price * (1 - self.stop_loss_pct)
                    self.take_profit_price = current_price * (1 + self.take_profit_pct)
                else:
                    self.stop_loss_price = current_price * (1 + self.stop_loss_pct)
                    self.take_profit_price = current_price * (1 - self.take_profit_pct)
                
                reward = 0.01
        
        elif action == 3 and self.position is not None:  # Exit position
            self._close_position(current_price, exit_reason or 'manual')
            reward = self._calculate_exit_reward(exit_reason)
        
        return reward
    
    def _close_position(self, exit_price, exit_reason):
        """Close position with proper P&L calculation"""
        if self.position == 'long':
            pnl_pct = (exit_price - self.entry_price) / self.entry_price
        else:
            pnl_pct = (self.entry_price - exit_price) / self.entry_price
        
        # Apply leverage
        leveraged_pnl_pct = pnl_pct * self.leverage
        
        # Calculate P&L amount
        pnl_amount = self.position_size * leveraged_pnl_pct
        exit_fee = self.position_size * self.fee_rate
        net_pnl = pnl_amount - exit_fee
        
        # Update balance
        self.balance += net_pnl
        
        # Record trade
        self.trades.append({
            'entry_price': self.entry_price,
            'exit_price': exit_price,
            'position': self.position,
            'pnl': net_pnl,
            'pnl_pct': pnl_pct,
            'leveraged_pnl_pct': leveraged_pnl_pct,
            'exit_reason': exit_reason,
            'leverage': self.leverage,
            'duration': self.current_step - self.entry_step
        })
        
        # Update streaks
        if net_pnl > 0:
            self.winning_streak += 1
            self.consecutive_losses = 0
        else:
            self.consecutive_losses += 1
            self.winning_streak = 0
        
        # Reset position
        self.position = None
        self.position_size = 0
        self.leverage = 1
    
    def _calculate_exit_reward(self, exit_reason):
        """Calculate reward based on exit reason"""
        if not self.trades:
            return 0
        
        last_trade = self.trades[-1]
        pnl_pct = last_trade['leveraged_pnl_pct']
        
        if exit_reason == 'liquidation':
            return -2.0
        elif exit_reason == 'stop_loss':
            return -0.5
        elif exit_reason == 'take_profit':
            return 1.0 + min(pnl_pct * 10, 1.0)
        elif exit_reason == 'trailing_stop':
            return 0.5 + min(pnl_pct * 10, 1.0)
        else:  # manual
            return np.tanh(pnl_pct * 20)
    
    def _calculate_position_size(self, action):
        """Dynamic position sizing based on market conditions"""
        size_pct = self.base_position_pct
        
        # Get current conditions
        current_vol = self.data['volatility'].iloc[self.current_step]
        rsi = self.data['rsi'].iloc[self.current_step]
        
        # Volatility adjustment
        if current_vol > 0:
            vol_percentile = stats.percentileofscore(
                self.data['volatility'].dropna().values, current_vol
            )
            if vol_percentile > 80:  # High volatility
                size_pct *= 0.7
            elif vol_percentile < 20:  # Low volatility
                size_pct *= 1.3
        
        # RSI adjustment
        if rsi < 30 or rsi > 70:  # Extremes
            size_pct *= 1.2
        
        # Streak adjustment
        if self.consecutive_losses >= 3:
            size_pct *= 0.5
        elif self.winning_streak >= 2:
            size_pct *= 1.1
        
        # Dynamic leverage
        if current_vol > 0:
            if rsi < 25 or rsi > 75:  # Strong extremes
                self.leverage = min(3, self.max_leverage)
            else:
                self.leverage = 2
        else:
            self.leverage = 2
        
        size_pct = np.clip(size_pct, self.min_position_pct, self.max_position_pct)
        return self.balance * size_pct
    
    def step(self, action):
        """Execute one step with proper liquidation checks"""
        # Get current bar data
        current_high = self.data['high'].iloc[self.current_step]
        current_low = self.data['low'].iloc[self.current_step]
        current_close = self.data['close'].iloc[self.current_step]
        
        # Check for intrabar events if we have a position
        if self.position is not None:
            # Check liquidation first (highest priority)
            liquidated, liq_price = self._check_intrabar_liquidation(
                self.entry_price, current_high, current_low, 
                self.position, self.leverage
            )
            
            if liquidated:
                self._close_position(liq_price, 'liquidation')
                reward = -2.0
            else:
                # Check stop loss
                if self.position == 'long' and current_low <= self.stop_loss_price:
                    self._close_position(self.stop_loss_price, 'stop_loss')
                    reward = -0.5
                elif self.position == 'short' and current_high >= self.stop_loss_price:
                    self._close_position(self.stop_loss_price, 'stop_loss')
                    reward = -0.5
                # Check take profit
                elif self.position == 'long' and current_high >= self.take_profit_price:
                    self._close_position(self.take_profit_price, 'take_profit')
                    reward = 1.0
                elif self.position == 'short' and current_low <= self.take_profit_price:
                    self._close_position(self.take_profit_price, 'take_profit')
                    reward = 1.0
                else:
                    # Normal action execution
                    reward = self._execute_action(action)
        else:
            # No position, normal action execution
            reward = self._execute_action(action)
        
        # Update equity tracking
        current_equity = self._calculate_equity()
        self.equity_curve.append(current_equity)
        self.peak_equity = max(self.peak_equity, current_equity)
        
        # Move to next step
        self.current_step += 1
        
        # Check if done
        done = (self.current_step >= len(self.data) - 1 or 
                current_equity <= self.initial_balance * 0.5)
        
        # Get next observation
        next_obs = self._get_observation() if not done else np.zeros_like(self._get_observation())
        
        info = {
            'equity': current_equity,
            'position': self.position,
            'trades': len(self.trades),
            'win_rate': self._calculate_win_rate()
        }
        
        return next_obs, reward, done, info
    
    # Other methods remain the same...
    def reset(self):
        """Reset environment"""
        self.current_step = max(200, int(0.1 * len(self.data)))
        self.balance = self.initial_balance
        self.position = None
        self.position_size = 0
        self.entry_price = 0
        self.entry_step = 0
        self.leverage = 1
        self.stop_loss_price = 0
        self.take_profit_price = 0
        
        self.trades = []
        self.equity_curve = [self.balance]
        self.peak_equity = self.balance
        self.consecutive_losses = 0
        self.winning_streak = 0
        
        return self._get_observation()
    
    def _calculate_equity(self):
        """Calculate current equity"""
        if self.position is None:
            return self.balance
        
        current_price = self.data['close'].iloc[self.current_step]
        
        if self.position == 'long':
            pnl_pct = (current_price - self.entry_price) / self.entry_price
        else:
            pnl_pct = (self.entry_price - current_price) / self.entry_price
        
        unrealized_pnl = self.position_size * pnl_pct * self.leverage
        return self.balance + unrealized_pnl
    
    def _calculate_win_rate(self):
        """Calculate win rate"""
        if not self.trades:
            return 0.5
        
        wins = sum(1 for t in self.trades if t['pnl'] > 0)
        return wins / len(self.trades)
    
    def get_valid_actions(self):
        """Get valid actions"""
        if self.position is None:
            return [0, 1, 2]  # HOLD, LONG, SHORT
        else:
            return [0, 3]  # HOLD, EXIT
    
    def _get_observation(self):
        """Get current observation"""
        # Market features
        market_features = self.data[self.feature_columns].iloc[self.current_step].values
        market_features = np.nan_to_num(market_features, 0.0)
        
        # Position features
        position_features = self._get_position_features()
        
        # Account features  
        account_features = self._get_account_features()
        
        observation = np.concatenate([market_features, position_features, account_features])
        return observation.astype(np.float32)
    
    def _get_position_features(self):
        """Get position features"""
        if self.position is None:
            return np.zeros(8)
        
        current_price = self.data['close'].iloc[self.current_step]
        
        is_long = float(self.position == 'long')
        is_short = float(self.position == 'short')
        
        if self.position == 'long':
            pnl_pct = (current_price - self.entry_price) / self.entry_price
        else:
            pnl_pct = (self.entry_price - current_price) / self.entry_price
        
        leveraged_pnl = pnl_pct * self.leverage
        bars_in_position = self.current_step - self.entry_step
        
        return np.array([
            is_long,
            is_short,
            pnl_pct,
            leveraged_pnl,
            bars_in_position / 100,
            self.leverage / self.max_leverage,
            self.position_size / self.initial_balance,
            min(bars_in_position / 48, 1)  # Time pressure
        ])
    
    def _get_account_features(self):
        """Get account features"""
        current_equity = self._calculate_equity()
        total_return = (current_equity - self.initial_balance) / self.initial_balance
        
        if self.peak_equity > 0:
            drawdown = (self.peak_equity - current_equity) / self.peak_equity
        else:
            drawdown = 0
        
        win_rate = self._calculate_win_rate()
        
        return np.array([
            self.balance / self.initial_balance,
            current_equity / self.initial_balance,
            total_return,
            drawdown,
            win_rate,
            min(self.winning_streak / 5, 1),
            min(self.consecutive_losses / 5, 1),
            len(self.trades) / 100
        ])

## 7. Enhanced Training System

In [ ]:
def train_enhanced_system(combined_data, episodes=500, save_models=True):
    """Train the enhanced multi-asset system"""
    
    # Validate data
    print("Validating data...")
    if combined_data.isnull().any().any():
        print("Warning: Found NaN values in data, cleaning...")
        combined_data = combined_data.ffill().bfill().fillna(0)
    
    # Check data range
    print(f"Data range: {combined_data['datetime'].min()} to {combined_data['datetime'].max()}")
    print(f"Total samples: {len(combined_data)}")
    
    # Split by time - ensure we have enough data
    total_days = (combined_data['datetime'].max() - combined_data['datetime'].min()).days
    
    if total_days < 90:
        print(f"Warning: Only {total_days} days of data available. Need more data for proper training.")
        # Use 70/30 split for limited data
        train_size = int(0.7 * len(combined_data))
        train_data = combined_data[:train_size].copy()
        test_data = combined_data[train_size:].copy()
    else:
        # Use date-based split for sufficient data
        split_date = combined_data['datetime'].max() - pd.Timedelta(days=30)
        train_data = combined_data[combined_data['datetime'] < split_date].copy()
        test_data = combined_data[combined_data['datetime'] >= split_date].copy()
    
    # Ensure we have data
    if len(train_data) < 1000:
        print("Error: Not enough training data. Please download more historical data.")
        return None, None, None
    
    print(f"Train data: {len(train_data)} samples ({train_data['datetime'].min()} to {train_data['datetime'].max()})")
    print(f"Test data: {len(test_data)} samples ({test_data['datetime'].min() if len(test_data) > 0 else 'None'} to {test_data['datetime'].max() if len(test_data) > 0 else 'None'})")
    
    # === Step 1: Train Enhanced XGBoost ===
    print("\n=== Training Enhanced XGBoost Direction Predictor ===")
    
    xgb_predictor = EnhancedDirectionPredictor(lookback=24, prediction_horizon=4)
    
    # Prepare XGBoost data
    X, y = xgb_predictor.prepare_data(train_data, feature_columns)
    
    if len(X) < 100:
        print("Error: Not enough samples for XGBoost training")
        return None, None, None
    
    # Train-validation split
    split_idx = int(0.8 * len(X))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    
    print(f"XGBoost train samples: {len(X_train)}")
    print(f"XGBoost validation samples: {len(X_val)}")
    
    # Train ensemble with validation
    xgb_predictor.train_ensemble(X_train, y_train, X_val, y_val, n_models=7)
    
    # Evaluate
    val_predictions = xgb_predictor.predict(X_val)
    val_accuracy = np.mean((val_predictions != 0))  # Non-zero predictions
    print(f"XGBoost validation signal rate: {val_accuracy:.2%}")
    
    # === Step 2: Initialize Enhanced PPO Agent ===
    print("\n=== Initializing Enhanced PPO Agent ===")
    
    # Create environment
    train_env = EnhancedCryptoFuturesEnv(train_data)
    
    # Get state dimensions
    initial_state = train_env.reset()
    state_dim = initial_state.shape[0]
    action_dim = train_env.action_space
    
    print(f"State dimension: {state_dim}")
    print(f"Action dimension: {action_dim}")
    
    # Initialize PPO agent
    ppo_agent = EnhancedPPOAgent(state_dim, action_dim, lr=3e-4)
    
    # === Step 3: Train PPO with Curriculum Learning ===
    print("\n=== Training Enhanced PPO Agent ===")
    
    episode_rewards = []
    episode_profits = []
    episode_trades = []
    episode_win_rates = []
    episode_sharpe_ratios = []
    
    best_sharpe = -np.inf
    best_profit = -np.inf
    patience = 0
    max_patience = 75
    
    # Curriculum learning parameters
    explore_rate = 0.3  # Start with high exploration
    explore_decay = 0.995
    min_explore = 0.05
    
    for episode in range(episodes):
        state = train_env.reset()
        episode_reward = 0
        
        # Prepare XGBoost features for the episode
        if train_env.current_step >= xgb_predictor.lookback:
            xgb_features = []
            for i in range(len(train_data) - train_env.current_step):
                if train_env.current_step + i >= xgb_predictor.lookback:
                    feat = train_data[feature_columns].iloc[
                        train_env.current_step + i - xgb_predictor.lookback:
                        train_env.current_step + i
                    ].values.flatten()
                    xgb_features.append(feat)
            
            if xgb_features:
                xgb_features = np.array(xgb_features)
                xgb_signals = xgb_predictor.predict(xgb_features)
            else:
                xgb_signals = None
        else:
            xgb_signals = None
        
        # Episode loop
        step_count = 0
        done = False
        episode_returns = []
        
        while not done:
            # Get XGBoost signal for current step
            if xgb_signals is not None and step_count < len(xgb_signals):
                current_xgb_signal = xgb_signals[step_count]
            else:
                current_xgb_signal = None
            
            # Get valid actions
            valid_actions = train_env.get_valid_actions()
            
            # Select action with exploration
            action = ppo_agent.act(state, current_xgb_signal, valid_actions, explore_rate)
            
            # Execute action
            next_state, reward, done, info = train_env.step(action)
            
            # Store reward and done
            ppo_agent.store_reward(reward, done)
            
            # Track returns for Sharpe ratio
            if len(train_env.equity_curve) > 1:
                ret = (train_env.equity_curve[-1] - train_env.equity_curve[-2]) / train_env.equity_curve[-2]
                episode_returns.append(ret)
            
            # Update state
            state = next_state
            episode_reward += reward
            step_count += 1
            
            if done:
                break
        
        # Update PPO
        ppo_agent.update(epochs=10)
        
        # Calculate episode metrics
        final_equity = info['equity']
        episode_profit = (final_equity / train_env.initial_balance - 1) * 100
        
        # Calculate Sharpe ratio
        if len(episode_returns) > 1:
            returns_std = np.std(episode_returns)
            if returns_std > 0:
                sharpe = np.sqrt(252 * 24) * np.mean(episode_returns) / returns_std  # Hourly to annual
            else:
                sharpe = 0
        else:
            sharpe = 0
        
        episode_rewards.append(episode_reward)
        episode_profits.append(episode_profit)
        episode_trades.append(info['trades'])
        episode_win_rates.append(info['win_rate'])
        episode_sharpe_ratios.append(sharpe)
        
        # Update exploration rate
        explore_rate = max(min_explore, explore_rate * explore_decay)
        
        # Early stopping based on Sharpe ratio
        if sharpe > best_sharpe:
            best_sharpe = sharpe
            patience = 0
            
            # Save best model
            if save_models and episode > 50:
                torch.save({
                    'ppo_state_dict': ppo_agent.network.state_dict(),
                    'episode': episode,
                    'sharpe': best_sharpe,
                    'profit': episode_profit
                }, 'best_ppo_model_enhanced.pth')
        else:
            patience += 1
        
        if episode_profit > best_profit:
            best_profit = episode_profit
        
        # Adaptive training phases
        if episode < 100:
            phase = "Phase 1: Exploration"
        elif episode < 300:
            phase = "Phase 2: Exploitation"
        else:
            phase = "Phase 3: Refinement"
        
        # Print progress
        if episode % 10 == 0:
            avg_profit = np.mean(episode_profits[-10:]) if len(episode_profits) >= 10 else episode_profit
            avg_trades = np.mean(episode_trades[-10:]) if len(episode_trades) >= 10 else info['trades']
            avg_win_rate = np.mean(episode_win_rates[-10:]) if len(episode_win_rates) >= 10 else info['win_rate']
            avg_sharpe = np.mean(episode_sharpe_ratios[-10:]) if len(episode_sharpe_ratios) >= 10 else sharpe
            
            print(f"Episode {episode} ({phase}) - Explore Rate: {explore_rate:.3f}")
            print(f"  Profit: {episode_profit:.2f}% | Avg: {avg_profit:.2f}% | Best: {best_profit:.2f}%")
            print(f"  Sharpe: {sharpe:.2f} | Avg: {avg_sharpe:.2f} | Best: {best_sharpe:.2f}")
            print(f"  Trades: {info['trades']} | Win Rate: {info['win_rate']:.2%}")
        
        # Check early stopping
        if patience >= max_patience and episode > 200:
            print(f"\nEarly stopping at episode {episode}")
            break
    
    # === Training Complete ===
    print("\n=== Training Complete ===")
    print(f"Best Sharpe Ratio: {best_sharpe:.2f}")
    print(f"Best Profit: {best_profit:.2f}%")
    print(f"Final average profit (last 50): {np.mean(episode_profits[-50:]):.2f}%")
    print(f"Final average Sharpe (last 50): {np.mean(episode_sharpe_ratios[-50:]):.2f}")
    print(f"Final average win rate (last 50): {np.mean(episode_win_rates[-50:]):.2%}")
    
    # Plot training progress
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Profits
    ax1.plot(episode_profits)
    ax1.axhline(y=0, color='r', linestyle='--')
    ax1.set_title('Episode Profits (%)')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Profit %')
    ax1.grid(True)
    
    # Sharpe Ratios
    ax2.plot(episode_sharpe_ratios)
    ax2.axhline(y=0, color='r', linestyle='--')
    ax2.axhline(y=1, color='g', linestyle='--', alpha=0.5)
    ax2.set_title('Episode Sharpe Ratios')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Sharpe Ratio')
    ax2.grid(True)
    
    # Win rates
    ax3.plot(episode_win_rates)
    ax3.axhline(y=0.5, color='r', linestyle='--')
    ax3.set_title('Win Rates')
    ax3.set_xlabel('Episode')
    ax3.set_ylabel('Win Rate')
    ax3.grid(True)
    
    # Moving averages
    window = 50
    if len(episode_profits) >= window:
        ma_profits = pd.Series(episode_profits).rolling(window).mean()
        ma_sharpe = pd.Series(episode_sharpe_ratios).rolling(window).mean()
        
        ax4.plot(ma_profits, label='MA Profit %', color='green')
        ax4.plot(ma_sharpe * 10, label='MA Sharpe x10', color='blue')
        ax4.set_title(f'{window}-Episode Moving Averages')
        ax4.set_xlabel('Episode')
        ax4.legend()
        ax4.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return ppo_agent, xgb_predictor, {
        'episode_profits': episode_profits,
        'episode_trades': episode_trades,
        'episode_win_rates': episode_win_rates,
        'episode_sharpe_ratios': episode_sharpe_ratios,
        'best_profit': best_profit,
        'best_sharpe': best_sharpe
    }

In [ ]:
# Train the enhanced system
print("Starting enhanced multi-asset training...")
ppo_agent, xgb_predictor, training_results = train_enhanced_system(combined_data, episodes=300)

## 8. Backtesting on Individual Assets

In [ ]:
def backtest_individual_asset(ppo_agent, xgb_predictor, asset_data, asset_name, plot_results=True):
    """Backtest on individual asset - FIXED for data handling"""
    
    print(f"\n=== Backtesting on {asset_name} ===")
    
    # Use only recent data for testing
    test_data = asset_data[asset_data['datetime'] >= '2023-01-01'].copy()
    
    # Check if we have test data
    if len(test_data) < 100:
        print(f"Warning: Only {len(test_data)} samples for {asset_name} testing. Need more recent data.")
        # Use last 30% of available data instead
        test_size = int(0.3 * len(asset_data))
        test_data = asset_data[-test_size:].copy()
    
    if len(test_data) < 50:
        print(f"Error: Not enough data for backtesting {asset_name}")
        return {
            'asset': asset_name,
            'total_return': 0,
            'sharpe_ratio': 0,
            'max_drawdown': 0,
            'total_trades': 0,
            'win_rate': 0,
            'final_equity': 10000,
            'equity_curve': [10000]
        }
    
    print(f"Test data range: {test_data['datetime'].min()} to {test_data['datetime'].max()}")
    print(f"Test samples: {len(test_data)}")
    
    # Create test environment
    test_env = EnhancedCryptoFuturesEnv(test_data)
    state = test_env.reset()
    
    # Prepare XGBoost features
    xgb_features = []
    for i in range(len(test_data)):
        if i >= xgb_predictor.lookback:
            feat = test_data[feature_columns].iloc[
                i - xgb_predictor.lookback:i
            ].values.flatten()
            xgb_features.append(feat)
    
    xgb_features = np.array(xgb_features) if xgb_features else None
    xgb_signals = xgb_predictor.predict(xgb_features) if xgb_features is not None else None
    
    # Trading loop
    actions = []
    prices = []
    positions = []
    equities = []
    
    # Detailed logging
    action_logs = []
    
    step_count = 0
    while True:
        # Get XGBoost signal
        if xgb_signals is not None and step_count < len(xgb_signals):
            current_xgb_signal = xgb_signals[step_count]
        else:
            current_xgb_signal = None
        
        # Get valid actions
        valid_actions = test_env.get_valid_actions()
        
        # Get action from PPO
        with torch.no_grad():
            action = ppo_agent.act(state, current_xgb_signal, valid_actions, explore_rate=0)
        
        # Execute action
        next_state, reward, done, info = test_env.step(action)
        
        # Record data
        actions.append(action)
        prices.append(test_data['close'].iloc[test_env.current_step - 1])
        positions.append(1 if test_env.position == 'long' else 
                        (-1 if test_env.position == 'short' else 0))
        equities.append(info['equity'])
        
        # Print first few trades
        if len(test_env.trades) > 0 and len(test_env.trades) <= 5:
            trade = test_env.trades[-1]
            print(f"[Trade #{len(test_env.trades)}] {trade['position'].upper()} "
                  f"Entry: ${trade['entry_price']:.2f} Exit: ${trade['exit_price']:.2f} "
                  f"P&L: ${trade['pnl']:.2f} ({trade['pnl_pct']*100:.2f}%) "
                  f"Leverage: {trade['leverage']}x Reason: {trade['exit_reason']}")
        
        # Update state
        state = next_state
        step_count += 1
        
        if done:
            break
    
    # Calculate metrics
    final_equity = equities[-1]
    total_return = (final_equity / test_env.initial_balance - 1) * 100
    
    # Calculate Sharpe ratio
    if len(equities) > 1:
        equity_returns = pd.Series(equities).pct_change().dropna()
        if len(equity_returns) > 0 and equity_returns.std() > 0:
            sharpe_ratio = np.sqrt(252 * 24) * equity_returns.mean() / equity_returns.std()
        else:
            sharpe_ratio = 0
    else:
        sharpe_ratio = 0
    
    # Calculate max drawdown
    if len(equities) > 0:
        equity_series = pd.Series(equities)
        running_max = equity_series.expanding().max()
        drawdown = (equity_series - running_max) / running_max
        max_drawdown = drawdown.min() * 100
    else:
        max_drawdown = 0
    
    # Print results
    print(f"\n=== {asset_name} Results ===")
    print(f"Total Return: {total_return:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_drawdown:.2f}%")
    print(f"Total Trades: {len(test_env.trades)}")
    print(f"Win Rate: {test_env._calculate_win_rate():.2%}")
    
    return {
        'asset': asset_name,
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'total_trades': len(test_env.trades),
        'win_rate': test_env._calculate_win_rate(),
        'final_equity': final_equity,
        'equity_curve': equities
    }

In [ ]:
# Backtest on each asset
results = {}
for symbol, data in multi_asset_data.items():
    asset_name = symbol.split('/')[0]
    results[asset_name] = backtest_individual_asset(
        ppo_agent, xgb_predictor, data, asset_name, plot_results=False
    )

In [ ]:
# Summary
print("\n=== PORTFOLIO SUMMARY ===")
print(f"{'Asset':<10} {'Return %':<12} {'Sharpe':<10} {'Max DD %':<12} {'Win Rate':<10} {'Trades':<10}")
print("-" * 70)

total_return = 0
for asset, res in results.items():
    print(f"{asset:<10} {res['total_return']:>10.2f}% {res['sharpe_ratio']:>8.2f} "
          f"{res['max_drawdown']:>10.2f}% {res['win_rate']:>8.2%} {res['total_trades']:>8}")
    total_return += res['total_return']

print("-" * 70)
print(f"{'AVERAGE':<10} {total_return/len(results):>10.2f}%")

print("\nTraining completed successfully! The model is now ready for live trading simulation.")